Il progetto trae ispirazione dalle tecniche sviluppate da Pangeanic e dal suo prodotto commerciale. Sebbene non ci sia l'intenzione di competere direttamente con Pangeanic, il loro approccio ha fornito preziosi spunti e chiarezza sulle possibili soluzioni per il problema del "NER Tagging e Anonimizzazione".

[Link alle soluzioni di Pangeanic per l'anonimizzazione](https://pangeanic.com/it/soluzioni-nlp/anonimizzazione)

al momento il codice gestisce i tag PER , ORG , LOC e MISC.
le entità di tipo PER ed ORG vengono riconosciute, accorpate e sostituite con un randomico iperinomio, viene gestito inoltre, tramite il tag nel testo, una molteplice occorrenza dell'entità che viene indicata con il numero assegnato.
Per quel che riguarda le LOC, vengono sostituiti dal tag generico [Location], sempre numerato, ma è possibile aggiungere più dettagli.
Il MISC invece viene solo segnalato al redattore, in quanto un tag generico non ha uno standard di gestione.

In [59]:
import pandas as pd
import numpy as np
import pickle


def read_tagging(file_name):
    path = "../data/" + "it" + "/tagging/" + file_name + ".conllu"
    data = pd.read_csv(path, sep="\t", quoting=3, names=["POSITION", "WORD", "TAG"])
    return data

In [60]:
import random


# Funzione di [Sostituzione] per ottenere o generare un pseudonimo per un'entità di tipo PERSON, ORGANIZATION o LOCATION
def get_pseudonym(
    entity_type, entity, pseudonym_counters, pseudonyms, generic_synonyms_or_hypernyms
):

    if entity_type in [
        "PER",
        "ORG",
        "LOC",
    ]:  # Verifica se l'entità è di uno dei tipi gestiti
        if entity in pseudonyms:  # Verifica se l'entità ha già un pseudonimo assegnato
            return pseudonyms[entity]  # Restituisce l'pseudonimo già assegnato
        else:
            # Se l'entità non ha ancora un pseudonimo, genera uno nuovo
            if entity not in pseudonym_counters[entity_type]:
                # Assegna un numero incrementale come identificativo dell'entità
                pseudonym_counters[entity_type][entity] = (
                    len(pseudonym_counters[entity_type]) + 1
                )
            # Costruisce l'pseudonimo con il formato corretto
            pseudonym = (
                f"[{entity_type.capitalize()}{pseudonym_counters[entity_type][entity]}]"
            )
            # Se è disponibile un elenco di sinonimi generici per il tipo di entità, aggiunge un sinonimo casuale
            if entity_type in generic_synonyms_or_hypernyms:
                pseudonym += (
                    f" {random.choice(generic_synonyms_or_hypernyms[entity_type])}"
                )
            # Memorizza l'pseudonimo per l'entità per evitare duplicazioni
            pseudonyms[entity] = pseudonym
            return pseudonym
    else:
        return entity  # Restituisce l'entità originale se non è di un tipo gestito

# Funzione di Maschera [Parola --> P****a]
def anonymize_mask(entity, tag, entity_counters):
    if tag not in entity_counters:
        entity_counters[tag] = {}

    if entity not in entity_counters[tag]:
        entity_counters[tag][entity] = len(entity_counters[tag]) + 1

    count = entity_counters[tag][entity]

    if len(entity) > 2:
        masked_entity = f"{entity[0]}{'*' * (len(entity) - 2)}{entity[-1]}"
    else:
        masked_entity = "*" * len(entity)

    return f"[{tag.capitalize()}{count}]{masked_entity}"


# Funzione di Lacuna [Parola --> ________]
def anonymize_gap(entity, tag, entity_counters):
    # Inizializziamo il contatore per il tipo di entità se non esiste
    if tag not in entity_counters:
        entity_counters[tag] = {}

    # Incrementiamo il contatore per l'entità specifica se è la prima volta che la incontriamo
    if entity not in entity_counters[tag]:
        entity_counters[tag][entity] = len(entity_counters[tag]) + 1

    # Costruiamo la rappresentazione della lacuna con "_" di dimensione fissa 8
    gap_entity = "_" * 8

    # Restituiamo la stringa formattata con il tag e l'indice dell'entità
    return f"[{tag.capitalize()}{entity_counters[tag][entity]}]{gap_entity}"





def get_user_choice():
    print("Scegli la tecnica di anonimizzazione:")
    print("1. Sostituzione")
    print("2. Lacuna")
    print("3. Mascheramento")
    choice = input("Inserisci il numero della tecnica scelta: ")
    print("")
    return choice

In [61]:
import pandas as pd


def anonymize_document(ner_df, generic_synonyms_or_hypernyms, technique):

    # Conversione di tutte le parole a stringhe
    ner_df["WORD"] = ner_df["WORD"].astype(str)

    # Dizionario che tiene traccia del numero di pseudonimi già assegnati per ogni tipo di entità
    pseudonym_counters = {
        "PER": {},  # Contatore per le entità di tipo PERSON
        "ORG": {},  # Contatore per le entità di tipo ORGANIZATION
        "LOC": {},  # Contatore per le entità di tipo LOCATION
        "MISC": 1,  # Contatore per le entità di tipo MISC (inizia da 1)
    }

    # Dizionario che mappa gli pseudonimi già assegnati a ciascuna entità
    pseudonyms = {}

    entity_counters = {}

    anonymized_text = ""
    current_tag = None
    buffer = []

    for index, row in ner_df.iterrows():
        word = row["WORD"]
        tag = row["TAG"]

        if tag.startswith("B-"):
            if buffer:
                if current_tag in ["PER", "ORG", "LOC"]:

                    if technique == "1":
                        pseudonym = get_pseudonym(
                            current_tag,
                            " ".join(buffer),
                            pseudonym_counters,
                            pseudonyms,
                            generic_synonyms_or_hypernyms,
                        )
                        anonymized_text += "[" + pseudonym + "]" + " "

                    elif technique == "2":  # Lacuna
                        anonymized_text += (
                            "["
                            + anonymize_gap(
                                " ".join(buffer), current_tag, entity_counters
                            )
                            + "]"
                            + " "
                        )

                    elif technique == "3":  # Mascheramento
                        anonymized_text += (
                            "["
                            + anonymize_mask(
                                " ".join(buffer), current_tag, entity_counters
                            )
                            + "]"
                            + " "
                        )

                elif current_tag == "MISC":
                    if technique == "2":
                        anonymized_text += (
                            "["
                            + anonymize_gap(
                                " ".join(buffer), current_tag, entity_counters
                            )
                            + "]"
                            + " "
                        )
                    else:
                        anonymized_text += (
                            "["
                            + anonymize_mask(
                                " ".join(buffer), current_tag, entity_counters
                            )
                            + "]"
                            + " "
                        )
                else:
                    anonymized_text += " ".join(buffer) + " "

                buffer = []

            current_tag = tag[2:]
            buffer.append(word)

        elif tag.startswith("I-") and current_tag == tag[2:]:
            buffer.append(word)
        else:
            if buffer:
                if current_tag in ["PER", "ORG", "LOC"]:
                    if technique == "1":
                        pseudonym = get_pseudonym(
                            current_tag,
                            " ".join(buffer),
                            pseudonym_counters,
                            pseudonyms,
                            generic_synonyms_or_hypernyms,
                        )
                        anonymized_text += "[" + pseudonym + "]" + " "

                    elif technique == "2":  # Lacuna
                        anonymized_text += (
                            "["
                            + anonymize_gap(
                                " ".join(buffer), current_tag, entity_counters
                            )
                            + "]"
                            + " "
                        )

                    elif technique == "3":  # Mascheramento
                        anonymized_text += (
                            "["
                            + anonymize_mask(
                                " ".join(buffer), current_tag, entity_counters
                            )
                            + "]"
                            + " "
                        )

                elif current_tag == "MISC":
                    if technique == "2":
                        anonymized_text += (
                            "["
                            + anonymize_gap(
                                " ".join(buffer), current_tag, entity_counters
                            )
                            + "]"
                            + " "
                        )
                    else:
                        anonymized_text += (
                            "["
                            + anonymize_mask(
                                " ".join(buffer), current_tag, entity_counters
                            )
                            + "]"
                            + " "
                        )
                else:
                    anonymized_text += " ".join(buffer) + " "
                buffer = []
                current_tag = None

            if word in [".", ",", "!", "?"]:
                anonymized_text = anonymized_text.rstrip() + word + " "
            else:
                anonymized_text += word + " "

    if buffer:
        if current_tag in ["PER", "ORG", "LOC"]:

            if technique == "1":
                pseudonym = get_pseudonym(
                    current_tag,
                    " ".join(buffer),
                    pseudonym_counters,
                    pseudonyms,
                    generic_synonyms_or_hypernyms,
                )
                anonymized_text += "[" + pseudonym + "]" + " "

            elif technique == "2":  # Lacuna
                anonymized_text += (
                    "["
                    + anonymize_gap(" ".join(buffer), current_tag, entity_counters)
                    + "]"
                    + " "
                )

            elif technique == "3":  # Mascheramento
                anonymized_text += (
                    "["
                    + anonymize_mask(" ".join(buffer), current_tag, entity_counters)
                    + "]"
                    + " "
                )

        elif current_tag == "MISC":
            if technique == "2":
                anonymized_text += (
                    "["
                    + anonymize_gap(" ".join(buffer), current_tag, entity_counters)
                    + "]"
                    + " "
                )
            else:
                anonymized_text += (
                    "["
                    + anonymize_mask(" ".join(buffer), current_tag, entity_counters)
                    + "]"
                    + " "
                )
        else:
            anonymized_text += " ".join(buffer) + " "

    anonymized_text = anonymized_text.strip()

    return anonymized_text

In [62]:
ner_df = read_tagging("viterbi_tag")
golden_df = read_tagging("golden_tag")

# Dizionario di sinonimi o iperonimie generici per PERSON e ORGANIZATION
generic_synonyms_or_hypernyms = {
    "PER": ["il soggetto", "l'individuo", "la persona", "il cittadino"],
    "ORG": ["la compagnia", "l'azienda", "l'entità", "l'organizzazione"],
    "LOC": ["Lo Stato", "Il Paese", "La Zona", "il Circondario"],
    # è possibile aggiungere elementi al dizionario
}

# sostituzione - maschera o lacuna
technique = get_user_choice()


vit_anonymized_document = anonymize_document(
    ner_df, generic_synonyms_or_hypernyms, technique
)
print("---HMM Taggin Anonimizzation------")
print(vit_anonymized_document)
print("")
golden_anonymized_document = anonymize_document(
    golden_df, generic_synonyms_or_hypernyms, technique
)
print("---Golden Taggin Anonimizzation------")
print(golden_anonymized_document)

Scegli la tecnica di anonimizzazione:
1. Sostituzione
2. Lacuna
3. Mascheramento

---HMM Taggin Anonimizzation------
Si stabilì ad [[Loc1] il Circondario] per la sua ammirazione nei confronti della letteratura tedesca ( aveva imparato la lingua in carcere ), specialmente per i romantici come [[Per1] la persona]. Anche i veicoli interplanetari utilizzano principalmente i razzi chimici, anche se alcuni hanno usato sperimentalmente con successo i nuovi propulsori ionici ( come ad esempio il satellite dell' [[Org1] la compagnia] SMART-1 ). Dal 1866 al 1869 egli frequentò il liceo di recente costruzione a [[Loc2] Il Paese], e nel 1870 egli passò gli esami sulle lingue classiche necessario per l' ammissione all' università. [[Misc1]U********a] militare venne costruita attraverso le colline che vanno da [[Loc3] Lo Stato] fino a [[Loc4] Il Paese]. Raggiunge la popolarità nel 1988 nel varietà televisivo " Cocco " di [[Org2] l'azienda] con il personaggio dell' automobilista " incazzato come una 

Valutazione delle performance

parametria classica come accuratezza, precisione(Di quelli trovati quali lo sono effettivamente) e recall(quanti non me ne sono scappati e sono passati in chiaro)